In [ ]:
library(RPostgreSQL)
library(twang)
library(Matching)
library(tidyverse)

In [ ]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [ ]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

In [ ]:
full_data <- dbGetQuery(con, "select * from merged_data")
head(full_data)

In [ ]:
dbDisconnect(con)
dbUnloadDriver(drv)

In [ ]:
to_factor <- function(x) {
    if(length(na.omit(unique(x))) <= 1) return(factor(x, levels = c(0, 1)))
    return(factor(x))
}

In [ ]:
factor_vars <- full_data %>%
    names %>%
    grep("flag|abnormal|icd|sedative", ., value = TRUE) %>%
    c("gender", "first_careunit", "echo", "vent", "vaso",
      "icu_adm_weekday", "icu_adm_hour", "mort_28_day")
factor_vars

In [ ]:
full_data <- full_data %>%
    mutate(echo_int = as.integer(echo)) %>%
    mutate(mort_28_day_int = as.integer(mort_28_day)) %>%
    mutate_at(factor_vars, to_factor)
full_data %>% pull(echo) %>% head

In [ ]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|icd|age|gender|weight|saps|sofa|elix_score|vent|vaso|icu_adm|careunit|sedative", .)) %>%
    discard(grepl("vs|lab", .) & grepl("flag", .) & !grepl("bnp|troponin|kinase", .)) %>%
    discard(grepl("bnp|troponin|kinase", .) & !grepl("flag", .)) %>%
    discard(grepl("min|max", .)) %>%
    discard(grepl("abnormal", .))
feature_names
length(feature_names)

In [ ]:
features <- full_data %>%
    select(!!!rlang::syms(feature_names))
head(features)

In [ ]:
label_name <- "echo"

In [ ]:
label <- full_data %>% pull(echo)
str(label)

In [ ]:
fml <- feature_names %>%
    c("echo", .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

In [ ]:
unweighted <- glm(as.formula(fml), data = full_data, family = binomial, na.action = na.exclude)
summary(unweighted)
exp(cbind(OR = coef(unweighted), confint(unweighted)))

In [ ]:
saveRDS(unweighted, file = file.path(data_dir, "multivariate_model.rds"))

In [ ]:
fml <- feature_names %>%
    paste(collapse = " + ") %>%
    sprintf("echo_int ~ %s", .)
fml

In [ ]:
echo_ps_ate <- ps(as.formula(fml),
                  data = full_data,
                  interaction.depth = 2,
                  shrinkage = 0.01,
                  perm.test.iters = 0,
                  estimand = "ATE",
                  verbose = FALSE,
                  stop.method = c("es.mean", "es.max", "ks.mean", "ks.max"),
                  n.trees = 100,
                  train.fraction = 0.8,
                  cv.folds = 3,
                  n.cores = 8)

In [ ]:
pred <- echo_ps_ate$ps$es.mean.ATE
full_data <- full_data %>% mutate(ps = pred)
ROCR::performance(ROCR::prediction(pred, label), "auc")@y.values %>% first

In [ ]:
ft_importance <- summary(echo_ps_ate$gbm.obj,
                         n.trees = echo_ps_ate$desc$es.mean.ATE$n.trees,
                         plot = FALSE)

In [ ]:
full_data <- full_data %>%
    mutate(ps_weight = get.weights(echo_ps_ate, stop.method = "es.mean"))

In [ ]:
saveRDS(full_data, file = file.path(data_dir, "full_data_ps.rds"))

In [ ]:
saveRDS(ft_importance, file = file.path(data_dir, "feature_importance.rds"))

In [ ]:
primary_ipw <- glm(mort_28_day ~ echo, data = full_data,
                   weights = full_data$ps_weight, family = binomial)
summary(primary_ipw)
exp(cbind(OR = coef(primary_ipw), confint(primary_ipw)))

In [ ]:
saveRDS(primary_ipw, file = file.path("ipw_model.rds"))

In [ ]:
ipw_svydesign <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [ ]:
saveRDS(ipw_svydesign, file = file.path(data_dir, "ipw_svydesign.rds"))

In [ ]:
fml <- feature_names %>%
    c(label_name, .) %>%
    paste(collapse = " + ") %>%
    sprintf("mort_28_day ~ %s", .)
fml

In [ ]:
logi <- svyglm(as.formula(fml),
               family = quasibinomial,
               design = design_echo_ps_ate)

In [ ]:
summary(logi)

In [ ]:
exp(cbind(OR = coef(logi), confint(logi)))

In [ ]:
saveRDS(logi, file = file.path(data_dir, "doubly_robust_all_model.rds"))